#Created by WMS, modified by MEK

The UCLA miniscope system produces unqiue timestamp logs for each miniscope and behavioral camera used in a session. This script:

1) loads timestamps for miniscope, minicam (single or multiple), and cleversys behavioral event output 

IMPORTANT: this script assumes that data directory is organized as .../session_date/trial_time/devices

### Step 0: load timestamp logs (either multiple or one per device per session)

In [1]:
import pandas as pd
import os
import re
import sys
#!{sys.executable} -m pip install openpyxl
import openpyxl #for loading cleversys xlsx
import pdb #for debugging
from matplotlib import pyplot as plt


#!{sys.executable} -m pip install xlrd
#import xlrd


#pip install yapf
#from yapf.yapflib.yafp_api import FormatCode

#identify imaging session directory
#sess_dir = "C://miniscopeData/5546/2023_01_01/13_17_27/01012023_PPTnodrug/"


sess_dir = "C://miniscopeData/5546/2022_12_15/13_56_20_record/12152022_cohabitation_5546/"
writeOutFolder = os.path.join(sess_dir + 'alignerOutput/')
writeCleversysCsvOutPath = os.path.join(writeOutFolder + 'cleversysevents.csv')
writeBorisCsvOutPath = os.path.join(writeOutFolder + 'borisevents.csv')

if not os.path.exists(writeOutFolder):
    os.makedirs(writeOutFolder)
frameRate = 50


#a = os.listdir(sess_dir)

#put all folders with HH_MM_SS format into list
#trials = []

#date_patt = "[0-2][0-9]_[0-5][0-9]_[0-5][0-9]"

#for i in a:
#    if re.search(date_patt, i):
#        trials.append(i)

#place in chronological order
#trials.sort()

In [2]:
#for loop that goes into each folder in trials, opens the MiniCam1, MiniCam2, and My_V4_Miniscope folders 
#and grabs the timestamp .csv from each

ms1 = "My_V4_Miniscope" #may need to change this with multi-scope setup
#ms2 = "My_V4_Miniscope2" #i don't actually yet know what the nomenclature will be for a second scope
mc1 = "MiniCam1"
mc2 = "MiniCam2"
cv = "cleversys"

devices = [ms1, mc1, cv] #ms2

#currently written assuming only cam 1 is the top-down

ts_files = []


ms1_ts_files = []
#ms2_ts_files = []
mc1_ts_files = []
mc2_ts_files = []
cv_event_files = []

#can nest these for loops likely
#for j in trials:
ms1_ts_path = os.path.join(sess_dir, ms1, "timeStamps.csv").replace("\\","/")
ms1_ts_files.append(ms1_ts_path)
    
#for k in trials:
#    ms2_ts_path = os.path.join(sess_dir, k, ms2, "timeStamps.csv").replace("\\","/")
#    ms2_ts_files.append(ms2_ts_path)

#for l in trials:
mc1_ts_path = os.path.join(sess_dir, mc1, "timeStamps.csv").replace("\\","/")
mc1_ts_files.append(mc1_ts_path)
    
#for m in trials:
#mc2_ts_path = os.path.join(sess_dir, j, mc2, "timeStamps.csv").replace("\\","/")
#mc2_ts_files.append(mc2_ts_path)

cv_ts_path = os.path.join(sess_dir, cv, "events1.xlsx").replace("\\","/")
cv_event_files.append(cv_ts_path)

### Step 1: concatenating each device's timestamp logs (not necessary if you only have one recording per device per session)

In [3]:
cv_event_files

['C://miniscopeData/5546/2022_12_15/13_56_20_record/12152022_cohabitation_5546/cleversys/events1.xlsx']

I assume the videoWriter function in the video concat script immediately places one videos' frames after the previous (ie frame 0 on second video becomes frame n+1 when n = last frame of first video). We should confirm this.

I also assume that Bento or other things importing concatenated files will want timestamps to be continuous integers throughout all the timestamps, not something like 1.123345, 2.12043034, where the prefix would denote which original file the timestamp came from

With those assumptions, the strategy in this step is to:

a) load first timestamp file from all streams (assuming your trials are ordered correctly, see above cell)

b) find maximum  frame # of each timestamp file (ms1_max = x, mc1_max = y, mc2_max = z) and save each as its own variable

c) find maximum timestamp of each timestamp file and compare to find the latest timestamp across the three streams' first video

d) save this max timestamp as its own variable and add 60000 (as a one minute spacer for a faux start time for the next video)

e) save each timestamp file into a new table

f) load next timestamp file for all streams

g) repeat b)

h) add the max frame from the previous file to each of the current frame numbers

i) repeat c) and d)

j) add the value calculated in d) to all timestamps in all loaded files

k) concatenate each timestamp file to previous file

l) repeat f-k for remaining files 

In [4]:
cv

'cleversys'

In [5]:
#IF RUNNING BORIS ONLY
#read ts files into dfs - any reason to make these a dictionary?
mscope1 = pd.read_csv(ms1_ts_files[0])
#mscope2 = pd.read_csv(ms2_ts_files[p])
mcam1 = pd.read_csv(mc1_ts_files[0])

In [6]:
#for p in range(len(trials)):
#read ts files into dfs - any reason to make these a dictionary?
mscope1 = pd.read_csv(ms1_ts_files[0])
#mscope2 = pd.read_csv(ms2_ts_files[p])
mcam1 = pd.read_csv(mc1_ts_files[0])
#mcam2 = pd.read_csv(mc2_ts_files[p])
cv = pd.read_excel(cv_event_files[0])

cv.rename(columns=cv.iloc[0]).drop(cv.index[0])
    
# if p == 0:
        #find max frame of each
ms1_maxF = mscope1['Frame Number'].max()
mc1_maxF = mcam1['Frame Number'].max()
#mc2_maxF = mcam2['Frame Number'].max()
    
#find max timestamp across all files, add 60000
ms1_maxTS = mscope1['Time Stamp (ms)'].max()
mc1_maxTS = mcam1['Time Stamp (ms)'].max()
#mc2_maxTS = mcam2['Time Stamp (ms)'].max()
#maxTSs = [ms1_maxTS, mc1_maxTS, mc2_maxTS]
maxTSs = [ms1_maxTS, mc1_maxTS]
        
#next_start_time = (max(maxTSs) + 60000)

#compile dfs into overall dfs
#ms1_concat = mscope1
#ms2_concat = mscope2
#mc1_concat = mcam1
#mc2_concat = mcam2


# Rename columns, add columns for frame start/stop
cv.columns= ['EventID','StartinSec','EndinSec','LengthinSec','EventType']
startTimes = cv['StartinSec']
startTimesInFrames = (startTimes * frameRate)
cv['startInFrames'] = startTimesInFrames

#pdb.set_trace()

endTimes = cv['EndinSec']
endTimesInFrames = (endTimes * frameRate)
cv['endInFrames'] = endTimesInFrames


cv = cv.tail(-1) #remove first header row

cv.startInFrames = cv.startInFrames.astype(int) #otherwise a combo of ints & floats for some reason
cv.endInFrames = cv.endInFrames.astype(int)
cv = cv.round({'startInFrames': 0}) #round to nearest integer
cv = cv.round({'endInFrames': 0})
cv
    
        

,EventID,StartinSec,EndinSec,LengthinSec,EventType,startInFrames,endInFrames
1,1,1,11.52,10.52,Area:Mouse 1 Center In All,50,576
2,2,1,11.52,10.52,Area:Mouse 2 Center In All,50,576
3,3,10.84,11.36,0.52,Social Contact [ 1 with 2 ] in Area All while ...,542,568
4,4,11.52,16.9,5.38,Area:Mouse 2 Center In All,576,844
5,5,11.52,16.9,5.38,Area:Mouse 1 Center In All,576,844
...,...,...,...,...,...,...,...
380,380,3370.1,3378.86,8.76,Area:Mouse 2 Center In All,168505,168943
381,381,3370.1,3378.86,8.76,Area:Mouse 1 Center In All,168505,168943
382,382,3376.86,3378.78,1.92,Social Contact [ 1 with 2 ] in Area All while ...,168843,168939
383,383,3378.86,3379.44,0.58,Area:Mouse 2 Center In All,168943,168972


In [8]:
#CLEVERSYS EVENT FILE TIMESTAMPS TO MINISCOPE FRAME INDEX

#for each behavior start and end time in frames, go into minicam and pull the corresponding timestamps
#using that timestamp, find nearest miniscope row
#write that info back into the cleversys dataframe and save it

#NOTE-the scopeFrame indices do not necessarily index Minian output appropriately
#e.g if there has been temporal downsampling


starts = cv.startInFrames
starts = starts.to_dict()
ends = cv.endInFrames
ends  = ends.to_dict()

timestampStart = {}
timestampEnd = {}
scopeFrameStart = {}
scopeFrameEnd = {}



for l in starts:
    indx = starts[l]
    #mc1_concat.iloc[[starts],1]
    timestampStart[l] = mcam1.iloc[indx,1] #grab the timestamp of that minicam frame
    indxEnd = ends[l]
    try: #to account for trying to grab one frame beyond what is in minicam
        timestampEnd[l] = mcam1.iloc[indxEnd,1] #grab the timestamp of that minicam frame
    except:
        timestampEnd[l] = mcam1.iloc[indxEnd-1,1]
    closest = mscope1.iloc[(mscope1['Time Stamp (ms)']-timestampStart[l]).abs().argsort()[:1]]
    scopeFrameStart[l] = round(closest.iloc[0,0] / 2)
    closest = mscope1.iloc[(mscope1['Time Stamp (ms)']-timestampEnd[l]).abs().argsort()[:1]]
    scopeFrameEnd[l] = round(closest.iloc[0,0] / 2)
    
    
#write back into cleversys dataframe
cv['startInMs'] = timestampStart.values()
cv['endInMs'] = timestampEnd.values()
cv['scopeFrameStart'] = scopeFrameStart.values() #frame # of the row with the nearest timestamp
cv['scopeFrameEnd'] = scopeFrameEnd.values()


# save updated dataframe out 
cv.to_csv(writeCleversysCsvOutPath)

cv




,EventID,StartinSec,EndinSec,LengthinSec,EventType,startInFrames,endInFrames,startInMs,endInMs,scopeFrameStart,scopeFrameEnd
1,1,1,11.52,10.52,Area:Mouse 1 Center In All,50,576,1059,12266,16,183
2,2,1,11.52,10.52,Area:Mouse 2 Center In All,50,576,1059,12266,16,183
3,3,10.84,11.36,0.52,Social Contact [ 1 with 2 ] in Area All while ...,542,568,11543,12095,172,180
4,4,11.52,16.9,5.38,Area:Mouse 2 Center In All,576,844,12266,17976,183,268
5,5,11.52,16.9,5.38,Area:Mouse 1 Center In All,576,844,12266,17976,183,268
...,...,...,...,...,...,...,...,...,...,...,...
380,380,3370.1,3378.86,8.76,Area:Mouse 2 Center In All,168505,168943,3590039,3599370,53466,53604
381,381,3370.1,3378.86,8.76,Area:Mouse 1 Center In All,168505,168943,3590039,3599370,53466,53604
382,382,3376.86,3378.78,1.92,Social Contact [ 1 with 2 ] in Area All while ...,168843,168939,3597239,3599286,53572,53603
383,383,3378.86,3379.44,0.58,Area:Mouse 2 Center In All,168943,168972,3599370,3599967,53604,53613


In [10]:
scopeFrameStart

{1: 16,
 2: 16,
 3: 172,
 4: 183,
 5: 183,
 6: 236,
 7: 268,
 8: 268,
 9: 320,
 10: 434,
 11: 514,
 12: 514,
 13: 643,
 14: 680,
 15: 680,
 16: 857,
 17: 874,
 18: 874,
 19: 1750,
 20: 1762,
 21: 1813,
 22: 1822,
 23: 1822,
 24: 1902,
 25: 1950,
 26: 1950,
 27: 2120,
 28: 2170,
 29: 2214,
 30: 2214,
 31: 2294,
 32: 2295,
 33: 2454,
 34: 2556,
 35: 2615,
 36: 2615,
 37: 2684,
 38: 2705,
 39: 2705,
 40: 2856,
 41: 3145,
 42: 3145,
 43: 3600,
 44: 3760,
 45: 3760,
 46: 3895,
 47: 3919,
 48: 3919,
 49: 4248,
 50: 4346,
 51: 4429,
 52: 4429,
 53: 4598,
 54: 5160,
 55: 5202,
 56: 5210,
 57: 5210,
 58: 5376,
 59: 5449,
 60: 5504,
 61: 5609,
 62: 5694,
 63: 5694,
 64: 5985,
 65: 6109,
 66: 6109,
 67: 6126,
 68: 6181,
 69: 6336,
 70: 6342,
 71: 6342,
 72: 6404,
 73: 6448,
 74: 6488,
 75: 6536,
 76: 6578,
 77: 6620,
 78: 6620,
 79: 6999,
 80: 7128,
 81: 7148,
 82: 7160,
 83: 7170,
 84: 7170,
 85: 7180,
 86: 7250,
 87: 7289,
 88: 7426,
 89: 7426,
 90: 7434,
 91: 7460,
 92: 7705,
 93: 7705,
 94: 7

In [26]:
# save updated dataframe out 
#cv.to_csv(writeCsvOutPath)
#type(boris1)

In [7]:
# BORIS EVENT TIMESTAMPS TO MINISCOPE FRAME INDEX
# same general procedure as last cell
borisInput = os.path.join(sess_dir + 'minicam1_compiled/output/aggregated_run2.tsv/')

boris1 = pd.read_csv(borisInput,sep='\t')


# get starting seconds, multiply by FPS to get frames, then round to integer
borisStarts = boris1['Start (s)']
borisStartsInFrames = (borisStarts * frameRate)
boris1['startFrameNum'] = borisStartsInFrames
boris1['startFrameNum'] = boris1['startFrameNum'].round(0).astype(int)

borisEnds = boris1['Stop (s)']
borisEndsInFrames = (borisEnds * frameRate)
boris1['endFrameNum'] = borisEndsInFrames
boris1['endFrameNum'] = boris1['endFrameNum'].round(0).astype(int)


boris1


starts = boris1.startFrameNum
starts = starts.to_dict()
ends = boris1.endFrameNum
ends  = ends.to_dict()

timestampStart = {}
timestampEnd = {}
scopeFrameStart = {}
scopeFrameEnd = {}



for l in starts:
    indx = starts[l]
    #mc1_concat.iloc[[starts],1]
    timestampStart[l] = mcam1.iloc[indx,1] #grab the timestamp of that minicam frame
    indxEnd = ends[l]
    try: #to account for trying to grab one frame beyond what is in minicam
        timestampEnd[l] = mcam1.iloc[indxEnd,1] #grab the timestamp of that minicam frame
    except:
        timestampEnd[l] = mcam1.iloc[indxEnd-1,1]
    closest = mscope1.iloc[(mscope1['Time Stamp (ms)']-timestampStart[l]).abs().argsort()[:1]]
    scopeFrameStart[l] = closest.iloc[0,0]
    closest = mscope1.iloc[(mscope1['Time Stamp (ms)']-timestampEnd[l]).abs().argsort()[:1]]
    scopeFrameEnd[l] = closest.iloc[0,0]
    
    
#write back into cleversys dataframe
#boris1['startInMs'] = timestampStart.values()
#boris1['endInMs'] = timestampEnd.values()
boris1['scopeFrameStart'] = scopeFrameStart.values() #frame # of the row with the nearest timestamp
boris1['scopeFrameEnd'] = scopeFrameEnd.values()


# save updated dataframe out 
boris1.to_csv(writeBorisCsvOutPath)

boris1




,Observation id,Observation date,Description,Media file,Total length,FPS,Subject,Behavior,Behavioral category,Modifiers,Behavior type,Start (s),Stop (s),Duration (s),Comment start,Comment stop,startFrameNum,endFrameNum,scopeFrameStart,scopeFrameEnd
0,5546_1215_135620,2022-12-26 20:58:00,First hour of cohousing,C:/miniscopeData/5546/2022_12_15/13_56_20_reco...,3379.44,50.0,No focal subject,noncontact investigation,Investigation,Both,STATE,8.80,10.30,1.50,NaN,NaN,440,515,140,164
1,5546_1215_135620,2022-12-26 20:58:00,First hour of cohousing,C:/miniscopeData/5546/2022_12_15/13_56_20_reco...,3379.44,50.0,No focal subject,noncontact investigation,Investigation,Initiating,STATE,10.54,13.88,3.34,NaN,NaN,527,694,168,220
2,5546_1215_135620,2022-12-26 20:58:00,First hour of cohousing,C:/miniscopeData/5546/2022_12_15/13_56_20_reco...,3379.44,50.0,No focal subject,noncontact investigation,Investigation,Initiating,STATE,25.18,32.80,7.62,NaN,NaN,1259,1640,400,520
3,5546_1215_135620,2022-12-26 20:58:00,First hour of cohousing,C:/miniscopeData/5546/2022_12_15/13_56_20_reco...,3379.44,50.0,No focal subject,noncontact investigation,Investigation,Initiating,STATE,33.88,36.14,2.26,NaN,NaN,1694,1807,538,574
4,5546_1215_135620,2022-12-26 20:58:00,First hour of cohousing,C:/miniscopeData/5546/2022_12_15/13_56_20_reco...,3379.44,50.0,No focal subject,grooming,Affiliative,Grooming self,STATE,48.28,59.40,11.12,NaN,NaN,2414,2970,766,942
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303,5546_1215_135620,2022-12-26 20:58:00,First hour of cohousing,C:/miniscopeData/5546/2022_12_15/13_56_20_reco...,3379.44,50.0,No focal subject,noncontact investigation,Investigation,Initiating,STATE,3328.98,3331.02,2.04,NaN,NaN,166449,166551,52813,52846
304,5546_1215_135620,2022-12-26 20:58:00,First hour of cohousing,C:/miniscopeData/5546/2022_12_15/13_56_20_reco...,3379.44,50.0,No focal subject,grooming,Affiliative,Grooming self,STATE,3338.70,3343.04,4.34,NaN,NaN,166935,167152,52967,53036
305,5546_1215_135620,2022-12-26 20:58:00,First hour of cohousing,C:/miniscopeData/5546/2022_12_15/13_56_20_reco...,3379.44,50.0,No focal subject,noncontact investigation,Investigation,Initiating,STATE,3349.78,3361.64,11.86,NaN,NaN,167489,168082,53143,53331
306,5546_1215_135620,2022-12-26 20:58:00,First hour of cohousing,C:/miniscopeData/5546/2022_12_15/13_56_20_reco...,3379.44,50.0,No focal subject,grooming,Affiliative,Grooming self,STATE,3370.38,3372.16,1.78,NaN,NaN,168519,168608,53470,53498


In [9]:
import pickle 


with open(os.path.join(writeOutFolder + 'forCaiman.pkl'), 'wb') as f:
    pickle.dump([cv, boris1], f)

In [7]:
# finally, load in Minian output and save out cleversys, boris, and minian files into a pkl


#get ready to load in Minian output
import holoviews as hv
import numpy as np
import xarray as xr

minian_path = "."
minian_ds_path = os.path.join(sess_dir, "My_V4_Miniscope\minian_output")

sys.path.append(minian_path)

from minian.utilities import (
    TaskAnnotation,
    get_optimal_chk,
    load_videos,
    open_minian,
    save_minian,
)



In [8]:
import pickle 

#load Minian output, convert it to df, drop rows with NaN
minian_ds = open_minian(minian_ds_path)
C_dropped = minian_ds['C_dropped']
C_dropped_df = C_dropped.to_pandas()
cellsKept = C_dropped_df.dropna()
#cellsKept

S_dropped = minian_ds['S_dropped']
S_dropped_df = S_dropped.to_pandas()
cellsKeptSpikes = S_dropped_df.dropna()
#cellsKeptSpikes

#implement cross-correlation here to do further merging

with open(os.path.join(writeOutFolder + 'minianOut.pkl'), 'wb') as f:
    pickle.dump([cellsKept, cellsKeptSpikes, cv, boris1], f)


#cellsKept = cellsKeptSpikes


